In [1]:
import json
import pandas as pd

def lines2df(lines):
    ex_dicts = []
    for line in lines:
        ex_dict = json.loads(line.strip())
        ex_dicts.append(ex_dict)
    df = pd.DataFrame.from_records(ex_dicts, columns=list(ex_dicts[0].keys()))
    return df

# lines = [l for l in open('/home/ubuntu/efs/lei/emerald_new/emerald.jsonl', 'r').readlines()]
lines = [l for l in open('/home/ubuntu/efs/lei/emerald_new/test.jsonl', 'r').readlines()]

emerald_df = lines2df(lines)

In [2]:
def concat_paragraph(sections):
    return '\n'.join(j for i in sections for j in i)
emerald_df['fulltext'] = emerald_df.sections.apply(concat_paragraph)
emerald_df['abstracttext'] = emerald_df.abstract_sections.apply(concat_paragraph)

In [3]:
def find_section_title_like(section_names, section_text, cuewords):
    text = []
    for sn, st in zip(section_names, section_text):
        sn = sn.lower()
        for cueword in cuewords:
            if cueword in sn:
                text.append(' '.join(st))
                break
    return ' '.join(text)

def find_section_title_not_like(section_names, section_text, cuewords):
    text = []
    for sn, st in zip(section_names, section_text):
        sn = sn.lower()
        positive = False
        for cueword in cuewords:
            if cueword in sn:
                positive = True
                break
        if not positive:
            text.append(' '.join(st))
    return ' '.join(text)


emerald_df['source_introduction'] = emerald_df.apply(lambda row: find_section_title_like(row['section_names'], row['sections'], ['intro', 'purpose']), axis=1)
emerald_df['source_design'] = emerald_df.apply(lambda row: find_section_title_like(row['section_names'], row['sections'], ['design', 'method', 'approach']), axis=1)
emerald_df['source_result'] = emerald_df.apply(lambda row: find_section_title_like(row['section_names'], row['sections'], ['result', 'find', 'discuss', 'analy']), axis=1)
emerald_df['source_conclusion'] = emerald_df.apply(lambda row: find_section_title_like(row['section_names'], row['sections'], ['conclu', 'future']), axis=1)
emerald_df['source_related'] = emerald_df.apply(lambda row: find_section_title_like(row['section_names'], row['sections'], ['related work', 'literat', 'background']), axis=1)

emerald_df['source_loo_introduction'] = emerald_df.apply(lambda row: find_section_title_not_like(row['section_names'], row['sections'], ['intro', 'purpose']), axis=1)
emerald_df['source_loo_design'] = emerald_df.apply(lambda row: find_section_title_not_like(row['section_names'], row['sections'], ['design', 'method', 'approach']), axis=1)
emerald_df['source_loo_result'] = emerald_df.apply(lambda row: find_section_title_not_like(row['section_names'], row['sections'], ['result', 'find', 'discuss', 'analy']), axis=1)
emerald_df['source_loo_conclusion'] = emerald_df.apply(lambda row: find_section_title_not_like(row['section_names'], row['sections'], ['conclu', 'future']), axis=1)
emerald_df['source_loo_related'] = emerald_df.apply(lambda row: find_section_title_not_like(row['section_names'], row['sections'], ['related work', 'literat', 'background']), axis=1)

In [4]:
emerald_df['source_IC'] = emerald_df['source_introduction'] + ' ' + emerald_df['source_conclusion']

In [5]:
# ('Purpose', 'Design/methodology/approach', 'Findings', 'Originality/value')
emerald_df['target_introduction'] = emerald_df.apply(lambda row: find_section_title_like(row['abstract_sections_names'], row['abstract_sections'],
                                                                                         ['Purpose'.lower()]), axis=1)
emerald_df['target_design'] = emerald_df.apply(lambda row: find_section_title_like(row['abstract_sections_names'], row['abstract_sections'],
                                                                                   ['Design/methodology/approach'.lower()]), axis=1)
emerald_df['target_findings'] = emerald_df.apply(lambda row: find_section_title_like(row['abstract_sections_names'], row['abstract_sections'],
                                                                                     ['Findings'.lower()]), axis=1)
emerald_df['target_originality'] = emerald_df.apply(lambda row: find_section_title_like(row['abstract_sections_names'], row['abstract_sections'],
                                                                                        ['Originality/value'.lower()]),axis=1)

In [6]:
emerald_df = emerald_df[['section_names', 'sections', 'fulltext', 'source_IC',
                         'source_introduction', 'source_design', 'source_result',
                         'source_conclusion', 'source_related',
                         'source_loo_introduction', 'source_loo_design',
                         'source_loo_result', 'source_loo_conclusion', 'source_loo_related',
                         'abstracttext',
                         'target_introduction', 'target_design', 'target_findings', 'target_originality']]
emerald_df.head(3)

,section_names,sections,fulltext,source_IC,source_introduction,source_design,source_result,source_conclusion,source_related,source_loo_introduction,source_loo_design,source_loo_result,source_loo_conclusion,source_loo_related,abstracttext,target_introduction,target_design,target_findings,target_originality
0,"[1. Introduction, 2. Writing qualitatively in ...",[[The increasing institutional pressure to pub...,The increasing institutional pressure to publi...,The increasing institutional pressure to publi...,The increasing institutional pressure to publi...,Our aim was to unearth data from like-minded c...,"Reading, and re-reading, the questions and ans...",Figure 3 pulls together the various patterned ...,,Research in the areas of entrepreneurship and ...,The increasing institutional pressure to publi...,The increasing institutional pressure to publi...,The increasing institutional pressure to publi...,The increasing institutional pressure to publi...,- The purpose of this paper is to report on a ...,- The purpose of this paper is to report on a ...,- Scholars who had published qualitative paper...,- Entrepreneurship scholars perceive their qua...,- Although there is a vigorous debate within t...
1,"[1. Introduction, 2. Literature review and hyp...",[[Today's manufacturing companies are far more...,Today's manufacturing companies are far more a...,Today's manufacturing companies are far more a...,Today's manufacturing companies are far more a...,The implementation of LM and FMS in US automot...,4.1 Measurement model assessment Four tests we...,"Ambiguity, uncertainty and complexity are just...","LM, also known as the Toyota Production System...","LM, also known as the Toyota Production System...",Today's manufacturing companies are far more a...,Today's manufacturing companies are far more a...,Today's manufacturing companies are far more a...,Today's manufacturing companies are far more a...,The purpose of this paper is to identify the m...,The purpose of this paper is to identify the m...,A survey questionnaire was developed based on ...,Lean and FMS are multi-dimensional philosophie...,This research empirically develops a framework...
2,"[Introduction, Methodology, Case description, ...","[[In the last 20 years, there has been a drama...","In the last 20 years, there has been a dramati...","In the last 20 years, there has been a dramati...","In the last 20 years, there has been a dramati...",Study design We chose an in-depth case study a...,Empirical patterns In the chain of pre-merger ...,,,Study design We chose an in-depth case study a...,"In the last 20 years, there has been a dramati...","In the last 20 years, there has been a dramati...","In the last 20 years, there has been a dramati...","In the last 20 years, there has been a dramati...",- The purpose of this paper is to examine how ...,- The purpose of this paper is to examine how ...,- Based on extensive document analysis and 35 ...,"- Spanning nearly a decade, the pre-merger pro...",- This is the first systematic in-depth study ...


In [7]:
emerald_df = emerald_df[['fulltext', 'abstracttext']]

In [8]:
# copy https://github.com/memray/bigsum
import numpy as np
from nltk.stem.porter import PorterStemmer
import spacy
from helper import rouge

stemmer = PorterStemmer()

from helper import rouge_score

def rouge1(evaluated_sentences, reference_sentences):
    return rouge_score.rouge_n(evaluated_sentences, reference_sentences, n=1, stopwords_removal=False, stemming=False, punctuations_removal=False)['f']

def rouge2(evaluated_sentences, reference_sentences):
    return rouge_score.rouge_n(evaluated_sentences, reference_sentences, n=2, stopwords_removal=False, stemming=False, punctuations_removal=False)['f']

def lcs(X, Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)

    # declaring the array for storing the dp values
    L = [[None] * (n + 1) for i in range(m + 1)]

    """Following steps build L[m + 1][n + 1] in bottom up fashion 
    Note: L[i][j] contains length of LCS of X[0..i-1] 
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])

                # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return L[m][n]

def oracle_extract(sents, summary_sents, rouge_method, lower=True, stemming=True):
    match_idx = []
    match_title = []
    match_scores = []

    # lowercase
    if lower:
        sents = [[w.lower() for w in sent] for sent in sents]
        summary_sents = [[w.lower() for w in sent] for sent in summary_sents]
    # stemming
    if stemming:
        sents = [[stemmer.stem(w) for w in sent] for sent in sents]
        summary_sents = [[stemmer.stem(w) for w in sent] for sent in summary_sents]

    for summary_id, summary_sent in enumerate(summary_sents):
        # rarely happens, number of summaries is larger than sents
        if summary_id >= len(sents):
            break

        match_score = [rouge_method(sent, summary_sent) for sent in sents]

        # remove the previously selected sents by setting their scores to 0
        for match_id in match_idx:
            match_score[match_id] = 0

        match_id = np.argmax(match_score)
        match_idx.append(match_id)
        match_scores.append(match_score[match_id])

    return match_idx, match_scores


def eval_rouge(sents, summary_sents, extract_sent_idx, number_to_cutoff=3, stopwords_removal=False, stemming=True, logger=None):
    rouge_ = rouge.Rouge(stopwords_removal=stopwords_removal, stemming=stemming)
    # extract_sent_idx = extract_sent_idx[: min(number_to_cutoff, len(extract_sent_idx))]
    # sort extracted sentences in the order of their appearance
    # extract_sent_idx = sorted(extract_sent_idx)
    extracted_sents = [sents[idx] for idx in extract_sent_idx if idx < len(sents)]
    hypothesis = ' '.join(' '.join(i) for i in extracted_sents)
    reference = summary_sents

    if hypothesis == None or reference == None or len(hypothesis.strip()) == 0 or len(reference.strip()) == 0:
        fscores = {k: 0.0 for k in metric_keys}
    else:
        scores = rouge_.get_scores(hypothesis, reference)
        fscores = {k: v['f'] for k, v in scores[0].items()}

    return fscores

In [9]:
from multiprocessing import Pool

# spacy_nlp = spacy.load('en_core_web_sm')
from pysbd.utils import PySBDFactory
spacy_nlp = spacy.blank('en')
spacy_nlp.add_pipe(PySBDFactory(spacy_nlp))

def oracle_score(section_text, abstract_text, rouge_method):
    source_sents = [[w.text for w in sent] for sent in spacy_nlp(section_text).sents]
    target_sents = [[w.text for w in sent] for sent in spacy_nlp(abstract_text).sents]

    extract_sent_idx, _ = oracle_extract(source_sents, target_sents, rouge_method)

    score = eval_rouge(source_sents,
                       ' '.join(w for sent in target_sents for w in sent),
                       extract_sent_idx)
    return score

In [10]:
from multiprocessing import Pool
from tqdm import tqdm
import sys
sys.setrecursionlimit(25000)

def func_lcs(i):
    return oracle_score(emerald_df['fulltext'][i], emerald_df['abstracttext'][i], lcs)
def func_rouge1(i):
    return oracle_score(emerald_df['fulltext'][i], emerald_df['abstracttext'][i], rouge1)
def func_rouge2(i):
    return oracle_score(emerald_df['fulltext'][i], emerald_df['abstracttext'][i], rouge2)

scores = {}

p = Pool(10)

for fn, f in (('rouge1', func_rouge1), ('rouge2', func_rouge2), ('lcs', func_lcs)):
    limit = emerald_df.shape[0]
    scores[fn] = []
    r = p.imap_unordered(f, list(range(limit)), chunksize=10)
    for score in tqdm(r, total=limit):
        scores[fn].append(score)


100%|██████████| 6000/6000 [25:13<00:00,  3.97it/s] 
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-9:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Process ForkPoolWorker-10:
Process ForkPoolWorker-6:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


In [10]:
import json
# json.dump(scores, open('oracle_score_testset_cmp_lcs_rouge.json', 'w'))
scores = json.load(open('oracle_score_testset_cmp_lcs_rouge.json', 'r'))

In [11]:
pd.DataFrame.from_records(scores['lcs'], columns=list(scores['lcs'][0].keys())).describe().loc['mean']

rouge-1    0.502001
rouge-2    0.276558
rouge-l    0.344522
Name: mean, dtype: float64

In [12]:
pd.DataFrame.from_records(scores['rouge1'], columns=list(scores['lcs'][0].keys())).describe().loc['mean']

rouge-1    0.585794
rouge-2    0.328724
rouge-l    0.448089
Name: mean, dtype: float64

In [13]:
pd.DataFrame.from_records(scores['rouge2'], columns=list(scores['lcs'][0].keys())).describe().loc['mean']

rouge-1    0.571981
rouge-2    0.344526
rouge-l    0.441829
Name: mean, dtype: float64

In [ ]:
from multiprocessing import Pool
from tqdm import tqdm
import sys
sys.setrecursionlimit(25000)


def rougeL(evaluated_sentences, reference_sentences):
    return rouge_score.rouge_l_summary_level(evaluated_sentences, reference_sentences, stopwords_removal=False, stemming=False, punctuations_removal=False)['f']

def func_rougeL(i):
    return oracle_score(emerald_df['fulltext'][i], emerald_df['abstracttext'][i], rougeL)
p = Pool(11)
fn, f = 'rougeL', func_rougeL
limit = emerald_df.shape[0]
scores[fn] = []
r = p.imap_unordered(f, list(range(limit)), chunksize=10)
for score in tqdm(r, total=limit):
    scores[fn].append(score)


 78%|███████▊  | 4661/6000 [2:33:58<38:31,  1.73s/it]  

In [ ]:
pd.DataFrame.from_records(scores['rougeL'], columns=list(scores['lcs'][0].keys())).describe().loc['mean']

In [ ]:
json.dump(scores, open('oracle_score_testset_cmp_lcs_rouge.json', 'w'))

In [18]:
# from multiprocessing import Pool
# import sys
# # fix a issue in rouge code
# sys.setrecursionlimit(25000)

# from tqdm import tqdm


# tasks = []
# for src_col in ('fulltext', 'source_introduction', 'source_design', 'source_result',
#                 'source_conclusion', 'source_related'
#                 'source_loo_introduction', 'source_loo_design', 'source_loo_result',
#                 'source_loo_conclusion', 'source_loo_related'):
#     for target_col in ('target_introduction', 'target_design', 'target_findings', 'target_originality'):
#         task_name = '_TO_'.join((src_col, target_col))
#         tasks.append((src_col, target_col, task_name))
        
# def func(a):
#     src_col, target_col, task_name = a
#     task_scores = []
#     for i in range(emerald_df.shape[0]):
#         src_text = emerald_df[src_col][i]
#         if not src_text.strip():
#             src_text = emerald_df['fulltext'][i]
#         r = oracle_score(src_text, emerald_df[target_col][i], None) ## TODO
#         task_scores.append(r)
#     return task_name, task_scores

# p = Pool(10)

# scores = {}
# r = p.imap_unordered(func, tasks, chunksize=1)
# for task_name, task_scores in tqdm(r, total=len(tasks)):
#     scores[task_name] = task_scores

##################################
# df_scores = []
# for col in scores:
#     df_score = pd.DataFrame.from_records(scores[col], columns=list(scores[col][0].keys())).describe()
#     df_score.columns = [col + '-' + i for i in df_score.columns]
#     df_scores.append(df_score)
# df_score = pd.concat(df_scores, axis=1)
# df_score
############################
# tmp = {}
# for col in df_score.columns:
#     src, tgt = col.split('_TO_')
#     if src not in tmp:
#         tmp[src] = {}
#     tmp[src][tgt] = df_score[col]['mean']
# df_cmp = pd.DataFrame(tmp)
# df_cmp.sort_index(key=lambda x: x.str[-1])